In [1]:
# General Libraries
import numpy as np
import pandas as pd
import os

# True Model
from scipy.integrate import odeint
from scipy.fft import fft

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras


# Data Preprocessing
from sklearn.preprocessing import StandardScaler





# My Modules
from wilkeXAI.data_generator import DataGenerator
import wilkeXAI.wilke_shap as fwg
    

In [2]:
"""
Define and Machine Learning Model
"""

def MLModel():
    opt = Adam(learning_rate=0.001, beta_1=0.7)
    loss='mse'
    model = Sequential([
        layers.Flatten(),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='sigmoid'),
        layers.Dense(32, activation='tanh'),
        layers.Dense(2)            
    ])
    model.compile(optimizer=opt, loss=loss)
    return model


In [35]:
suffix = "Run_1"
generator = DataGenerator()
# Generate the data
X_train, y_train = generator.generate(num_samples=int(1e4))
X_test, y_test = generator.generate(num_samples=int(1e3))

Generating Data…: 100%|████████████████| 1000/1000 [00:14<00:00, 67.49it/s]


In [48]:
X_train.loc[:,'x0':'t']

,x0,v0,t
0,-1.015976,0.701215,0.0
1,-1.015976,0.701215,0.2
2,-1.015976,0.701215,0.4
3,-1.015976,0.701215,0.6
4,-1.015976,0.701215,0.8
...,...,...,...
4999995,0.437671,0.225635,99.0
4999996,0.437671,0.225635,99.2
4999997,0.437671,0.225635,99.4
4999998,0.437671,0.225635,99.6


In [50]:
scaler = StandardScaler()

scaler.fit(X_train.loc[:,'x0':'t'].values)
scaler.transform(X_train.loc[:,'x0':'t'].values, copy=False)
scaler.transform(X_test.loc[:,'x0':'t'].values, copy = False)

array([[-0.32969714,  0.21075447, -1.72859016],
       [-0.32969714,  0.21075447, -1.72166195],
       [-0.32969714,  0.21075447, -1.71473373],
       ...,
       [ 1.07581826, -0.90680433,  1.71473373],
       [ 1.07581826, -0.90680433,  1.72166195],
       [ 1.07581826, -0.90680433,  1.72859016]])

In [61]:
X_test.values[:,:3]

array([[-0.32969714,  0.21075447, -1.72859016],
       [-0.32969714,  0.21075447, -1.72166195],
       [-0.32969714,  0.21075447, -1.71473373],
       ...,
       [ 1.07581826, -0.90680433,  1.71473373],
       [ 1.07581826, -0.90680433,  1.72166195],
       [ 1.07581826, -0.90680433,  1.72859016]])

In [51]:
model = MLModel()
true_model = fwg.TrueModel(scaler, X_test)

In [52]:
# Train Network
# Model Weights Path
checkpoint_path = "Networks//training//"+suffix+"cp1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks = [cp_callback,
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15),
             tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15)]


history=model.fit(X_train, y_train, steps_per_epoch=None, epochs=3, 
                  validation_split=0.2, batch_size=20364, shuffle=True, callbacks=callbacks, verbose=2)

Epoch 1/3
197/197 - 25s - loss: 0.0895 - val_loss: 0.0678

Epoch 00001: saving model to Networks\training
Epoch 2/3
197/197 - 25s - loss: 0.0688 - val_loss: 0.0681

Epoch 00002: saving model to Networks\training
Epoch 3/3
197/197 - 24s - loss: 0.0677 - val_loss: 0.0662

Epoch 00003: saving model to Networks\training


In [57]:
X_test.columns[:4]

Index(['x0', 'v0', 't', 'rand'], dtype='object')

In [8]:
models = {'ml': model, 
         'true': true_model}
       
for exp_type in ['shap', 'lime', 'analytic']:
    explainer_curr = fwg.wilke_explainer(models, X_train, X_test, y_test, explainer_type=exp_type, tolerance = 1)
    explainer_curr.eval_explainer().to_csv("Results/"+exp_type+"/individual/"+suffix+".csv")
    explainer_curr.aggregate().to_csv("Results/"+exp_type+"/aggregate/"+suffix+".csv")

0it [00:00, ?it/s]

IndexError: list index out of range

In [16]:
explainer_test = fwg.wilke_explainer(models, X_train, X_test, y_test, explainer_type='shap', tolerance = 0.1)

In [26]:
choose_vals(0,9)

array([ True,  True,  True, ...,  True,  True,  True])

In [25]:
def choose_vals(i, num_features):
        """
            Chooses the data points on which the explainer will be evaluated for
            a given feature. First, removes any points where the prediction is
            further than tolerance from the true value. Second, takes values in
            a band of thickness tol around 0 for all features not currently being
            evaluated on. Third, chooses a random subset of length num_vals of 
            these values. This function should never be called outside of this class.
            
            Parameters
            ----------
            i : int, index of the feature being explained
            feature : str, the feature being explained
            num_features : the total number of features
            
            Returns
            ----------
            data_arr : pandas.DataFrame, array of the chosen values.
        """
        vals = np.abs(np.linalg.norm((models['ml']).predict(X_test), axis=1) - 
                      np.linalg.norm(y_test, axis=1))
        data_arr = X_test.iloc[np.where(vals < 0.1)]
        where__ = np.ones_like(data_arr.values[:,i], dtype=bool)
        for j in range(1,num_features):
            where__ = np.multiply(where__, np.abs(data_arr.values[:,(i + j)%num_features])<self.tol)
        return where__
    

In [ ]:
def choose_data(i, num_features):
        """
            Chooses the data points on which the explainer will be evaluated for
            a given feature. First, removes any points where the prediction is
            further than tolerance from the true value. Second, takes values in
            a band of thickness tol around 0 for all features not currently being
            evaluated on. Third, chooses a random subset of length num_vals of 
            these values. This function should never be called outside of this class.
            
            Parameters
            ----------
            i : int, index of the feature being explained
            feature : str, the feature being explained
            num_features : the total number of features
            
            Returns
            ----------
            data_arr : pandas.DataFrame, array of the chosen values.
        """
        vals = np.abs(np.linalg.norm((models['ml']).predict(X_test), axis=1) - 
                      np.linalg.norm(y_test, axis=1))
        
        print(vals)
        data_arr = self.test_data.iloc[np.where(vals < self.tolerance)]
        where__ = np.ones_like(data_arr.values[:,i], dtype=bool)
        for j in range(1,num_features):
            where__ = np.multiply(where__, np.abs(data_arr.values[:,(i + j)%num_features])<self.tol)
        data_arr = data_arr.iloc[where__]
        data_arr = data_arr.iloc[np.sort(
            np.random.choice(data_arr.shape[0], np.min([self.num_vals, data_arr.shape[0]]), replace=False))]
        return data_arr

In [24]:
where__ = np.ones_like(data_arr.values[:,i], dtype=bool)

NameError: name 'data_arr' is not defined

In [11]:
params = {'alpha' : [-1],'beta' : [1], 'gamma' :[0.37], 'delta' : [1], 'omega' : [1.2]}

In [15]:
for beta in params['gamma']:
    print(beta)

0.37


In [34]:
X_test.set_index(['alpha', 'beta', 'gamma', 'delta', 'omega'])

AttributeError: 'numpy.ndarray' object has no attribute 'set_index'

In [32]:
X_test = scaler.inverse_transform(X_test)

In [33]:
X_test

array([[-1.95477679, -0.70577194,  0.        , ...,  0.37      ,
         1.        ,  1.2       ],
       [-1.95477679, -0.70577194,  0.2       , ...,  0.37      ,
         1.        ,  1.2       ],
       [-1.95477679, -0.70577194,  0.4       , ...,  0.37      ,
         1.        ,  1.2       ],
       ...,
       [ 0.12764016,  0.75095404, 99.4       , ...,  0.37      ,
         1.        ,  1.2       ],
       [ 0.12764016,  0.75095404, 99.6       , ...,  0.37      ,
         1.        ,  1.2       ],
       [ 0.12764016,  0.75095404, 99.8       , ...,  0.37      ,
         1.        ,  1.2       ]])